In [1]:
# Erasmus+ ICCT project (2018-1-SI01-KA203-047081)

# Toggle cell visibility

from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Toggle cell visibility <a href="javascript:code_toggle()">here</a>.''')
display(tag)

# Hide the code completely

# from IPython.display import HTML
# tag = HTML('''<style>
# div.input {
#     display:none;
# }
# </style>''')
# display(tag)

## Dinamica della velocità di una automobile

Questo esempio mostra la dinamica della velocità di un'auto scrivendola prima utilizzando le equazioni differenziali e quindi riformulandole in forma di stato.

La dinamica della velocità di un'auto è governata da due forze principali: trazione e resistenza all'aria. I dati necessari per descrivere il sistema sono: massa dell'auto $m = 1000$ kg; coppia massima del motore $\tau_{\text{max}}=150$ Nm; raggio delle ruote $r=25$ cm (si assume per semplicità che il rapporto di trasmissione $\eta$ sia costante e pari a 4:1 motore:ruota); coefficiente della resistenza aerodinamica $b = 60$ Ns / m (resistenza all'aria modellata come funzione lineare della velocità $v$).

Si presume che la dinamica del motore, dal comando dell'acceleratore $t_{r}$ (limitato in $[0, 1]$) alla frazione ottenuta della coppia massima $\tau_{\%}$, possa essere modellata come:

$$
\dot{\tau}_{\%}=-\frac{1}{2}\tau_{\%}+\frac{1}{2}t_{r};
$$

ovvero come un sistema lineare del primo ordine con costante di tempo $T=2$ e guadagno unitario.

### Equazioni differenziali
Con l'equazione di Newton ($F=ma$) è possibile scrivere l'equazione differenziale che descrive il moto dell'auto:

$$
m\dot{v}= \frac{\tau_{\text{max}}\eta}{r}\tau_{\%}-bv,
$$

dove la coppia generata dal motore è ottenuta moltiplicando $\tau_{\%}$ per la coppia massima $\tau_{\text{max}}$. Il comportamento del sistema viene quindi descritto dal sistema delle equazioni differenziali:

$$
\begin{cases}
m\dot{v}= \frac{\tau_{\text{max}}\eta}{r}\tau_{\%}-bv \\
\dot{\tau}_{\%}=-\frac{1}{2}\tau_{\%}+\frac{1}{2}t_{r}
\end{cases}
$$

### Forma di stato
Poiché le due equazioni differenziali sono entrambe _equazioni differenziali del primo ordine_, il numero di stati necessari per descrivere completamente il comportamento del sistema è due. Definito il vettore di stato come
$x=\begin{bmatrix}x_1&x_2\end{bmatrix}^T=\begin{bmatrix}v&\tau_{\%}\end{bmatrix}^T$ e trattando $t_r$ come input, nello spazio degli stati le equazioni diventano:

$$
\dot{x}=\underbrace{\begin{bmatrix}-\frac{b}{m}&\frac{\tau_{\text{max}}\eta}{mr}\\0&-\frac{1}{2}\end{bmatrix}}_{A}x+\underbrace{\begin{bmatrix}0\\\frac{1}{2}\end{bmatrix}}_{B}t_r \\
$$

### Come usare questo notebook?

Gioca con i parametri del sistema e prova a rispondere alle seguenti domande:
- Il semplice modello di automobile di questo esempio può raggiungere una velocità infinita? Perché?
- La velocità massima dipende dalla massa dell'auto? Perché?
- Un valore di accelerazione negativo ($t_r$) è significativo per questo modello? Perché?

In [2]:
#Preparatory Cell 

%matplotlib notebook
import control
import numpy
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt

%matplotlib inline
#print a matrix latex-like
def bmatrix(a):
     """Returns a LaTeX bmatrix - by Damir Arbula (ICCT project)

     :a: numpy array
     :returns: LaTeX bmatrix as a string
     """
     if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
     lines = str(a).replace('[', '').replace(']', '').splitlines()
     rv = [r'\begin{bmatrix}']
     rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
     rv +=  [r'\end{bmatrix}']
     return '\n'.join(rv)


# Display formatted matrix: 
def vmatrix(a):
    if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{vmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{vmatrix}']
    return '\n'.join(rv)


#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

             
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass

In [3]:
#define matrixes
C = numpy.matrix([[1,0],[0,1]])
D = numpy.matrix([[0],[0]])
X0 = matrixWidget(2,1)


m = widgets.FloatSlider(
    value=1000,
    min=400,
    max=2000,
    step=1,
    description='m [kg]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
eta = widgets.FloatSlider(
    value=4,
    min=0.8,
    max=10.0,
    step=0.1,
    description='$\eta$:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
tau_max = widgets.FloatSlider(
    value=150,
    min=50,
    max=900,
    step=1,
    description=r'$\tau_{\text{max}}$ [Nm]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
b_air = widgets.FloatSlider(
    value=60,
    min=0,
    max=200,
    step=1,
    description=r'$b$ [Ns/m]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
u = widgets.FloatSlider(
    value=0.5,
    min=0,
    max=1,
    step=0.01,
    description='',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
omega = widgets.FloatSlider(
    value=5,
    min=0,
    max=10.0,
    step=0.1,
    description='',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

In [4]:
#create dummy widget 
DW = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))

#create button widget
START = widgets.Button(
    description='Click!',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Clicca se hai cambiato le condizioni iniziali',
    icon='check'
)
                       
def on_start_button_clicked(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW.value> 0 :
        DW.value = -1
    else: 
        DW.value = 1
    pass
START.on_click(on_start_button_clicked)

#define type of ipout 
SELECT = widgets.Dropdown(
    options=[('impulso','impulse'), ('gradino','step'), ('sinusoide','sinusoid')],
    value='step',
    description='',
    disabled=False
)

In [5]:
def main_callback(X0, m, eta, tau_max, b_air, u, selu, omega, DW):
    r = 0.25 # m
    a = numpy.matrix([[-b_air/m,tau_max*eta/m/r],[0,-1/2]])
    b = numpy.matrix([[0],[1/2]])

    eig = numpy.linalg.eig(a)
    sys = sss(a,b,C,D)
    
    if min(numpy.real(abs(eig[0]))) != 0:
        T = numpy.linspace(0,10/min(numpy.real(abs(eig[0]))),1000)
    else:
        if max(numpy.real(abs(eig[0]))) != 0:
            T = numpy.linspace(0,10/max(numpy.real(abs(eig[0]))),1000)
        else:
            T = numpy.linspace(0,180,1000)
        
    if selu == 'impulse': #selu
        U = [0 for t in range(0,len(T))]
        U[0] = u
        y = control.forced_response(sys,T,U,X0)
    if selu == 'step':
        U = [u for t in range(0,len(T))]
        y = control.forced_response(sys,T,U,X0)
    if selu == 'sinusoid':
        U = u*numpy.sin(omega*T)
        y = control.forced_response(sys,T,U,X0)
    
    fig=plt.figure(num=1,figsize=[15, 4])
    fig.add_subplot(121)
    plt.plot(T,y[1][0])
    plt.grid()
    plt.xlabel('t [s]')
    plt.ylabel('velocità [m/s]')
    
    fig.add_subplot(122)
    plt.plot(T,y[1][1])
    plt.grid()
    plt.xlabel('t [s]')
    plt.ylabel(r'$\tau_\%$')
    
    #display(Markdown('The A matrix is: $%s$ and the eigenvalues are: $%s$' % (bmatrix(a),eig[0])))

#create a graphic structure to hold all widgets 
alltogether =  widgets.VBox([widgets.HBox([widgets.VBox([m,
                                           eta,
                                           tau_max,
                                           b_air]),
                            widgets.HBox([widgets.VBox([widgets.Label('Seleziona il tipo di input:',border=3),
                                                        widgets.Label('$t_r$:',border=3),
                                                        widgets.Label('omega [rad/s]:',border=3)]),
                                          widgets.VBox([SELECT,u,omega])])]),
                             widgets.HBox([widgets.Label('Stato iniziale X0:',border=3),X0,
                                           widgets.Label(' Clicca se hai cambiato le condizioni iniziali:',border=3),START])])

out = widgets.interactive_output(main_callback,{'X0':X0, 'm': m, 'eta': eta, 'tau_max': tau_max, 'b_air': b_air, 'u': u, 'selu': SELECT, 'omega':omega, 'DW':DW})
#out.layout.height = '300px'
display(out,alltogether)

Output()

In [6]:
#create dummy widget 2
DW2 = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))
DW2.value = -1

#create button widget
START2 = widgets.Button(
    description='Mostra le risposte',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Clicca per vedere le risposte',
    icon='check'
)
                       
def on_start_button_clicked2(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW2.value> 0 :
        DW2.value = -1
    else: 
        DW2.value = 1
    pass
START2.on_click(on_start_button_clicked2)

def main_callback2(DW2):
    if DW2 > 0:
        display(Markdown(r'''>Risposte:
                            >- Si, ma solo se il coefficiente di attrito $b$ è nullo.
                            >- No, la massa dell'auto cambia solo il tempo necessario a raggiungere una certa velocità; quindi varia l'accelerazione, non la velocità massima. 
                            >- Si, potrebbe essere utilizzato per modellare la frenata. '''))
    else:
        display(Markdown(''))

#create a graphic structure to hold all widgets 
alltogether2 =  widgets.VBox([START2])

out2 = widgets.interactive_output(main_callback2,{'DW2':DW2})
#out.layout.height = '300px'
display(out2,alltogether2)

Output()